In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/My Drive/DL_MP2')

In [ ]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as ps
from PIL import Image
import json
import random
from sklearn.model_selection import KFold
from pathlib import Path
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import pickle
import cv2
from fnmatch import fnmatch
from torch.utils.data import Dataset, ConcatDataset

Preparing the CSV first

In [ ]:
def CSV_Work(input_drive,output_drive,only_frontal):
  df=pd.read_csv(input_drive)
  df['subject_id']=df['Path'].apply(lambda x: int(Path(x).parent.parent.name[7:])).astype(str)
  df=df[df.Sex.isin(['Male', 'Female'])]
  def age_bins(x):
    if 0<=x<40: return '18-40'
    elif 40<=x<60: return '40-60'
    elif 60<=x<80: return '60-80'
    else: return '80-'
  df['Age']=df['Age'].apply(age_bins)
  copy=df['subject_id']
  df=df.drop(columns=['subject_id']).replace([[None],"NaN",-1,"[False]","[True]", "[ True]"],
                                                   [0,0,0,0,1,1])
  df['subject_id']=copy.astype(str)
  df['Sex']=np.where(df['Sex']=='Female','F',df['Sex'])
  df['Sex']=np.where(df['Sex']=='Male','M',df['Sex'])
  # No need to make changes to match MIMIC
  df['path']=df['Path']
  #df['path']=df['Path'].astype(str).apply(lambda x: os.path.join('/CheXpert-v1.0-small/',x))
  #print(df.head())
  df['frontal']=(df['Frontal/Lateral']=='Frontal')
  if only_frontal:
    df=df[df['frontal']]
  df['study_id']=df['path'].apply(lambda x: x[x.index('patient'):x.rindex('/')])
  df=df[['subject_id','path','Sex',"Age",'frontal','study_id','No Finding','Atelectasis','Cardiomegaly','Pleural Effusion','Pneumonia','Pneumothorax','Consolidation','Edema']]
  df.reset_index(drop=True).to_csv(output_drive,index=False)
  return df
  #return df[['subject_id','path','Sex',"Age",'frontal','study_id','No Finding','Atelectasis','Cardiomegaly','Pleural Effusion','Pneumonia','Pneumothorax','Consolidation','Edema']]

In [ ]:
train_pd=CSV_Work('CheXpert-v1.0-small/train.csv','CheXpert-v1.0-small/processed_train.csv',False)
val_pd=CSV_Work('CheXpert-v1.0-small/valid.csv','CheXpert-v1.0-small/processed_valid.csv',False)

In [ ]:
valid_pd=pd.read_csv('CheXpert-v1.0-small/processed_valid.csv')
train_pd=pd.read_csv('CheXpert-v1.0-small/processed_valid.csv')

In [ ]:
transform=transforms.Compose([transforms.RandomHorizontalFlip(),
                               transforms.RandomRotation(10),
                               transforms.RandomResizedCrop(size=224,scale=(0.75,1.0)),
                               transforms.ToTensor(),
                               transforms.Normalize([0.5],[0.2])])

In [ ]:
def image_Work(root,transform):
  data={}
  pattern="*.jpg"
  for path,subdirs,files in os.walk(root):
    for name in files:
      if fnmatch(name,pattern):
        location=os.path.join(path,name)
        image=Image.open(location)
        image=transform(image)
        data[location]=image
  return data



In [ ]:
train_set=image_Work('CheXpert-v1.0-small/train',transform)
val_set=image_Work('CheXpert-v1.0-small/valid',transform)

In [ ]:
t=open("train_set.pkl","wb")
pickle.dump(train_set,t)
t.close()
v=open("val_set.pkl","wb")
pickle.dump(val_set,v)
v.close()